# Setup

## Imports

In [2]:
import time
import numpy as np
import pytest
from JuliaSet import calculate_z_serial_purepython
from test_juliaset import test_calc_pure_python


## Julia Set Values

In [3]:
# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

# Exercise 1: PyTest with the Julia Set Code

## 1.1: Testing with PyTest Framework

### Implemented Test

In [3]:
def test_calc_pure_python(desired_width=1000, max_iterations=300):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    print("new sum: ", sum(output))
    assert sum(output) == 33219980

### Test Results

In [4]:
! pytest test_juliaset.py

============================= test session starts =============================
platform win32 -- Python 3.12.8, pytest-8.3.4, pluggy-1.5.0
rootdir: c:\Users\phoeb\OneDrive\Documents\KTH\HPC\data-structures-methods
plugins: anyio-4.6.2.post1
collected 1 item

test_juliaset.py .                                                       [100%]

============================== 1 passed in 4.92s ==============================


## 1.2: Testing with Varying Iterations & Grid Points

To test with varying iterations and grid points, we would use a parameterized test. This is shown through `@pytest.mark.parameterize()`, and the parameters are argnames and argvalues for the string names and values respectively. This allows us to assert different expected values based on the parameters and test each of them. 

### Implemented Test

In [5]:
import pytest
@pytest.mark.parameterize('desired_width, max_iterations, expected', [(1000, 300, 33219980)])
def test_calc_pure_python_param(desired_width, max_iterations, expected):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    print("new sum: ", sum(output))
    assert sum(output) == expected

### Test Results

In [6]:
! pytest test_juliaset_param.py

============================= test session starts =============================
platform win32 -- Python 3.12.8, pytest-8.3.4, pluggy-1.5.0
rootdir: c:\Users\phoeb\OneDrive\Documents\KTH\HPC\data-structures-methods
plugins: anyio-4.6.2.post1
collected 1 item

test_juliaset_param.py .                                                 [100%]

============================== 1 passed in 4.84s ==============================


# Exercise 2: Python DGEMM Benchmark Operation

## 2.1: Implementing DGEMM with NumPy

In [7]:
a_np = np.ones((3, 3), dtype=np.double)
b_np = np.ones((3, 3), dtype=np.double)
c_np = np.ones((3, 3), dtype=np.double)
expected_np = np.array([[4,4,4],[4,4,4],[4,4,4]])

def dgemm_numpy(a, b, c):
  N = a.shape[0]
  for i in range(N):
    for j in range(N):
      for k in range(N):
        c[i][j] = c[i][j] + a[i][k] * b[k][j]
  return c

## 2.2: Unit Test for DGEMM

### Test

In [8]:
def test_dgemm():
  assert np.array_equal(dgemm_numpy(a_np,b_np,c_np), expected_np) == True

### Test Results

In [9]:
! pytest test_dgemm.py

============================= test session starts =============================
platform win32 -- Python 3.12.8, pytest-8.3.4, pluggy-1.5.0
rootdir: c:\Users\phoeb\OneDrive\Documents\KTH\HPC\data-structures-methods
plugins: anyio-4.6.2.post1
collected 1 item

test_dgemm.py .                                                          [100%]

============================== 1 passed in 0.11s ==============================


In [10]:
import numpy as np
import time

def dgemm_numpy(a, b, c):
    N = a.shape[0]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

def measure_execution_time(matrix_size, runs=5):
    times = []
    for _ in range(runs):
        # Initialize matrices with random values
        a = np.ones((matrix_size, matrix_size), dtype=np.double)
        b = np.ones((matrix_size, matrix_size), dtype=np.double)
        c = np.zeros((matrix_size, matrix_size), dtype=np.double)

        # Measure execution time
        start_time = time.time()
        dgemm_numpy(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)

    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Testing with different matrix sizes
matrix_sizes = [10, 50, 100, 200]  # You can increase these sizes for larger benchmarks
results = []

for size in matrix_sizes:
    avg_time, std_dev = measure_execution_time(size)
    results.append((size, avg_time, std_dev))

# Print results
print("Matrix Size | Avg. Time (s) | Std. Dev. (s)")
print("------------------------------------------")
for size, avg, std in results:
    print(f"{size:^11} | {avg:^13.6f} | {std:^12.6f}")

Matrix Size | Avg. Time (s) | Std. Dev. (s)
------------------------------------------
    10      |   0.000400    |   0.000490  
    50      |   0.067988    |   0.009108  
    100     |   0.488158    |   0.006455  
    200     |   4.259108    |   0.131729  


Implementation with array: 

In [11]:
import array

def dgemm_array(a, b, c, N):
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

# Helper function to create 2D arrays
def create_2d_array(size):
    return [array.array('d', [0] * size) for _ in range(size)]

Implementation using list 

In [12]:
def dgemm_list(a, b, c):
    N = len(a)
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

## Performance Measurement Code LIST, ARRAY, NumPy

The code handles and benchmarks multiple implementations sequentially in the same script. If the system runs other background processes or has load variations during the benchmarking process, these might disproportionately affect one implementation’s results. Furthermore, timing overhead might be added due to the use of lambdas for the array implementation. 

In [13]:
import numpy as np
import time

# Measure execution time for different implementations
def measure_time_implementation(implementation, matrix_size, runs=5):
    times = []
    for _ in range(runs):
        if implementation == "list":
            # Initialize matrices as lists
            a = [[1.0] * matrix_size for _ in range(matrix_size)]
            b = [[1.0] * matrix_size for _ in range(matrix_size)]
            c = [[0.0] * matrix_size for _ in range(matrix_size)]
            func = dgemm_list
        elif implementation == "array":
            # Initialize matrices as arrays
            a = create_2d_array(matrix_size)
            b = create_2d_array(matrix_size)
            c = create_2d_array(matrix_size)
            for i in range(matrix_size):
                for j in range(matrix_size):
                    a[i][j] = b[i][j] = 1.0
            func = lambda a, b, c: dgemm_array(a, b, c, matrix_size)
        elif implementation == "numpy":
            # Initialize matrices as NumPy arrays
            a = np.ones((matrix_size, matrix_size), dtype=np.double)
            b = np.ones((matrix_size, matrix_size), dtype=np.double)
            c = np.zeros((matrix_size, matrix_size), dtype=np.double)
            func = dgemm_numpy
        else:
            raise ValueError("Unsupported implementation")

        # Measure time
        start_time = time.time()
        func(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)

    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Compare implementations
implementations = ["list", "array", "numpy"]
matrix_sizes = [10, 50, 100, 200, 400]  # You can extend these sizes as needed
results = []

for size in matrix_sizes:
    for impl in implementations:
        avg_time, std_dev = measure_time_implementation(impl, size)
        results.append((impl, size, avg_time, std_dev))

# Print results
print("Implementation | Matrix Size | Avg. Time (s) | Std. Dev. (s)")
print("-----------------------------------------------------------")
for impl, size, avg, std in results:
    print(f"{impl:^13} | {size:^11} | {avg:^13.6f} | {std:^12.6f}")

Implementation | Matrix Size | Avg. Time (s) | Std. Dev. (s)
-----------------------------------------------------------
    list      |     10      |   0.000000    |   0.000000  
    array     |     10      |   0.000000    |   0.000000  
    numpy     |     10      |   0.000855    |   0.000439  
    list      |     50      |   0.008387    |   0.003362  
    array     |     50      |   0.018179    |   0.002997  
    numpy     |     50      |   0.068648    |   0.010827  
    list      |     100     |   0.061487    |   0.005221  
    array     |     100     |   0.118956    |   0.006789  
    numpy     |     100     |   0.524995    |   0.005175  
    list      |     200     |   0.503359    |   0.023612  
    array     |     200     |   0.808957    |   0.008360  
    numpy     |     200     |   3.961747    |   0.066757  
    list      |     400     |   4.285885    |   0.025221  
    array     |     400     |   7.265540    |   0.079518  
    numpy     |     400     |   32.548607   |   0.272

How does the computational performance, e.g., the std, vary with increasing the size of the matrices, and why so?

### 1. Average Time Increases with Matrix Size:
As the matrix size increases, the average execution time grows significantly for all implementations (list, array, and NumPy).
This is expected because the computational complexity of matrix multiplication is $O(N^3)$, meaning the number of operations grows cubically with the matrix size N.
### 2.Standard Deviation Trends:
For small matrices (e.g., 10x10), the std. dev. is relatively small for all implementations.
As the matrix size increases, the std. dev. generally increases, but the rate of increase varies across implementations.
NumPy exhibits a higher std. dev. compared to lists and arrays, especially for larger matrices (e.g., 400x400).
### 3. Relative Performance:
For small matrices, the list implementation is the fastest, followed by arrays, and then NumPy. This trend stays consistent over increasing matrix size. 
For larger matrices, NumPy becomes significantly slower compared to lists and arrays, despite being optimized for numerical computations. This is likely due to NumPy beeing optimized for vectorized operations, but the implementation of DGEMM in this exercise uses explicit Python loops (for i, for j, for k). This negates the benefits of NumPy's vectorization and introduces significant overhead.

# Task 2.4 
To calculate the FLOPS/s (floating-point operations per second), we first need to determine the number of floating-point operations (FLOPs) performed in the DGEMM operation. Then we divide the total number of FLOPs by the average time taken.

In DGEMM, the operation is: 
$C[i][j] = C[i][j] + A[i][k] \times B[k][j] $
- The operation involves:
	- 1 multiplication:  A[i][k] \times B[k][j] 
	- 1 addition:  C[i][j] + \text{result of multiplication} 
	- Total FLOPs per iteration of the innermost loop: 2 FLOPs

The total number of iterations across the nested loops is $N^3$, where $N$ is the matrix size. Therefore:

$\text{Total FLOPs} = 2 \times N^3$


The FLOPS/s for a given implementation is:

$\text{FLOPS/s} = \frac{\text{Total FLOPs}}{\text{Avg. Time (s)}}$


In [1]:
def calculate_flops(matrix_size, avg_time):
    # Calculate the total number of FLOPs
    total_flops = 2 * (matrix_size ** 3)
    # Calculate FLOPS/s
    flops_per_second = total_flops / avg_time
    return total_flops, flops_per_second

# Timing results
results = [
    ("list", 10, 0.000101),
    ("array", 10, 0.000199),
    ("numpy", 10, 0.000768),
    ("list", 50, 0.015739),
    ("array", 50, 0.010727),
    ("numpy", 50, 0.047675),
    ("list", 100, 0.048073),
    ("array", 100, 0.085522),
    ("numpy", 100, 0.372809),
    ("list", 200, 0.384661),
    ("array", 200, 0.668303),
    ("numpy", 200, 2.929246),
    ("list", 400, 3.115510),
    ("array", 400, 5.645596),
    ("numpy", 400, 24.057626),
]

# Calculate FLOPS/s for each result
print("Implementation | Matrix Size | Total FLOP(s)  | FLOPS/s")
print("---------------------------------------------------------")
for impl, size, avg_time in results:
    total_flops, flops_per_second = calculate_flops(size, avg_time)
    print(f"{impl:^13} | {size:^11} | {total_flops:^11} | {flops_per_second:.2e}")

Implementation | Matrix Size | Total FLOP(s)  | FLOPS/s
---------------------------------------------------------
    list      |     10      |    2000     | 1.98e+07
    array     |     10      |    2000     | 1.01e+07
    numpy     |     10      |    2000     | 2.60e+06
    list      |     50      |   250000    | 1.59e+07
    array     |     50      |   250000    | 2.33e+07
    numpy     |     50      |   250000    | 5.24e+06
    list      |     100     |   2000000   | 4.16e+07
    array     |     100     |   2000000   | 2.34e+07
    numpy     |     100     |   2000000   | 5.36e+06
    list      |     200     |  16000000   | 4.16e+07
    array     |     200     |  16000000   | 2.39e+07
    numpy     |     200     |  16000000   | 5.46e+06
    list      |     400     |  128000000  | 4.11e+07
    array     |     400     |  128000000  | 2.27e+07
    numpy     |     400     |  128000000  | 5.32e+06


### Comparing to Theoretical Peak Performance

Substituding values for Mac Pro 2020 M1 chip: 
	•	Clock Frequency = 3.2 GHz = 3.2 × 10⁹ cycles/second.
	•	FLOPs per Cycle = 2 (double precision).
	•	Number of High-Performance Cores = 4.

$\text{Peak FLOPS/s} = 3.2 \times 10^9 \times 2 \times 4 = 25.6 \, \text{GFLOPS (double precision)} $

For single precision (32-bit floating-point), NEON can handle 4 single-precision numbers per cycle per core:

$\text{Peak FLOPS/s (single precision)} = 3.2 \times 10^9 \times 4 \times 4 = 51.2 \, \text{GFLOPS}$


The theoretical peak performance of the Apple M1 chip at 25.6 GFLOPS (double precision) far exceeds the actual measured performance, indicating that real-world DGEMM implementations face significant bottlenecks, such as memory bandwidth limitations, system overhead, and non-ideal utilization of vectorized instructions. This gap highlights the challenges in achieving the full computational potential of the hardware, as practical workloads rarely operate at maximum efficiency due to these constraints.


## TODO: Write about caching - why some avg times are 0, second time running code values become 0, etc

In [6]:
import numpy as np
import time

# Custom DGEMM implementation
def dgemm_custom(a, b, c):
    N = a.shape[0]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

# Function to measure execution time
def measure_execution_time(func, a, b, c, runs=5):
    times = []
    for _ in range(runs):
        start_time = time.time()
        func(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)
    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Function to benchmark both implementations
def benchmark(matrix_size, runs=5):
    # Initialize matrices
    a = np.random.rand(matrix_size, matrix_size).astype(np.double)
    b = np.random.rand(matrix_size, matrix_size).astype(np.double)
    c_custom = np.zeros((matrix_size, matrix_size), dtype=np.double)
    c_numpy = np.zeros((matrix_size, matrix_size), dtype=np.double)

    # Measure custom DGEMM
    avg_time_custom, std_dev_custom = measure_execution_time(dgemm_custom, a, b, c_custom, runs)

    # Measure NumPy matmul
    avg_time_numpy, std_dev_numpy = measure_execution_time(lambda x, y, z: np.matmul(x, y, out=z), a, b, c_numpy, runs)

    # Print results
    print(f"Matrix Size: {matrix_size}x{matrix_size}")
    print(f"Custom DGEMM - Avg. Time: {avg_time_custom:.6f}s, Std. Dev.: {std_dev_custom:.6f}s")
    print(f"NumPy matmul - Avg. Time: {avg_time_numpy:.6f}s, Std. Dev.: {std_dev_numpy:.6f}s")
    print()

# Run benchmark for different matrix sizes
matrix_sizes = [10, 50, 100, 200]
for size in matrix_sizes:
    benchmark(size)

Matrix Size: 10x10
Custom DGEMM - Avg. Time: 0.001273s, Std. Dev.: 0.001109s
NumPy matmul - Avg. Time: 0.000000s, Std. Dev.: 0.000000s

Matrix Size: 50x50
Custom DGEMM - Avg. Time: 0.091253s, Std. Dev.: 0.009511s
NumPy matmul - Avg. Time: 0.000000s, Std. Dev.: 0.000000s

Matrix Size: 100x100
Custom DGEMM - Avg. Time: 0.727348s, Std. Dev.: 0.019342s
NumPy matmul - Avg. Time: 0.000000s, Std. Dev.: 0.000000s

Matrix Size: 200x200
Custom DGEMM - Avg. Time: 5.743056s, Std. Dev.: 0.188019s
NumPy matmul - Avg. Time: 0.001190s, Std. Dev.: 0.001702s



In [5]:
import numpy as np
import time

# Custom DGEMM implementation
def dgemm_custom(a, b, c):
    N = a.shape[0]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                c[i][j] += a[i][k] * b[k][j]
    return c

# Function to measure execution time
def measure_execution_time(func, a, b, c, runs=5):
    times = []
    for _ in range(runs):
        start_time = time.time()
        func(a, b, c)
        end_time = time.time()
        times.append(end_time - start_time)
    avg_time = np.mean(times)
    std_dev = np.std(times)
    return avg_time, std_dev

# Function to calculate FLOPS
def calculate_flops(matrix_size, avg_time):
    total_flops = 2 * (matrix_size ** 3)  # 2 FLOPs per iteration (1 multiply + 1 add)
    flops_per_second = total_flops / avg_time if avg_time > 0 else 0
    return total_flops, flops_per_second

# Function to benchmark both implementations and compute FLOPS
def benchmark(matrix_size, runs=5):
    # Initialize matrices
    a = np.random.rand(matrix_size, matrix_size).astype(np.double)
    b = np.random.rand(matrix_size, matrix_size).astype(np.double)
    c_custom = np.zeros((matrix_size, matrix_size), dtype=np.double)
    c_numpy = np.zeros((matrix_size, matrix_size), dtype=np.double)

    # Measure custom DGEMM
    avg_time_custom, std_dev_custom = measure_execution_time(dgemm_custom, a, b, c_custom, runs)
    total_flops_custom, flops_custom = calculate_flops(matrix_size, avg_time_custom)

    # Measure NumPy matmul
    avg_time_numpy, std_dev_numpy = measure_execution_time(lambda x, y, z: np.matmul(x, y, out=z), a, b, c_numpy, runs)
    total_flops_numpy, flops_numpy = calculate_flops(matrix_size, avg_time_numpy)

    # Print results
    print(f"Matrix Size: {matrix_size}x{matrix_size}")
    print(f"Custom DGEMM - Avg. Time: {avg_time_custom:.6f}s, Std. Dev.: {std_dev_custom:.6f}s, FLOPS: {flops_custom:.2e}")
    print(f"NumPy matmul - Avg. Time: {avg_time_numpy:.6f}s, Std. Dev.: {std_dev_numpy:.6f}s, FLOPS: {flops_numpy:.2e}")
    print()

# Run benchmark for different matrix sizes
matrix_sizes = [10, 50, 100, 200]
for size in matrix_sizes:
    benchmark(size)

Matrix Size: 10x10
Custom DGEMM - Avg. Time: 0.001040s, Std. Dev.: 0.001327s, FLOPS: 1.92e+06
NumPy matmul - Avg. Time: 0.000000s, Std. Dev.: 0.000000s, FLOPS: 0.00e+00

Matrix Size: 50x50
Custom DGEMM - Avg. Time: 0.085787s, Std. Dev.: 0.003505s, FLOPS: 2.91e+06
NumPy matmul - Avg. Time: 0.000000s, Std. Dev.: 0.000000s, FLOPS: 0.00e+00

Matrix Size: 100x100
Custom DGEMM - Avg. Time: 0.655219s, Std. Dev.: 0.011645s, FLOPS: 3.05e+06
NumPy matmul - Avg. Time: 0.000000s, Std. Dev.: 0.000000s, FLOPS: 0.00e+00

Matrix Size: 200x200
Custom DGEMM - Avg. Time: 5.218712s, Std. Dev.: 0.117734s, FLOPS: 3.07e+06
NumPy matmul - Avg. Time: 0.000000s, Std. Dev.: 0.000000s, FLOPS: 0.00e+00



NumPy’s matmul not only leverages the full capabilities of the M1 chip but also surpasses the theoretical peak due to highly optimized, hardware-specific operations. This demonstrates that for computationally intensive tasks, using optimized libraries like BLAS is critical for achieving performance close to or beyond theoretical limits.

# Exercise 3 - Experiment with Python Debugger Reflection

By using a debugger, the programmer is able to use many functionalities that help answer questions about the code that might not be obvious when running through the code. Some of the advantages include being able to go through each line of code and decide to either enter into the function call to help trace your code, or step over it to get to a different faulty spot.  While you are running in the debugger, you can also inspect the variables values, set breakpoints, and look at how the stack is ordered. If you want, you can also see the files in your code that are running and what they look like. This ability to inspect the program state during execution can be very helpful. If something is going wrong, the programmer can see exactly which function or even which instruction caused the state to deviate from what they expect. Some challenges that we found were that it was quite difficult to get started with the debugger. While the installation and running of it was simple, it was difficult to know exactly what to do in order to inspect our code properly. There are many capabilities that the debugger offers, it is just difficult to know which ones to use and where. It's a useful tool once a programmer is proficient with it but there is a learning curve.

# Bonus Exercise
### Performance Analysis and Optimization of the Game of Life Code


## Task B.1

In [17]:
# pip install pylint
! pylint game_of_life.py

************* Module game_of_life
game_of_life.py:11:31: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:24:38: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:25:38: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:59:28: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:64:70: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:66:62: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:67:62: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:68:74: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:95:0: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:100:0: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:127:18: C0303: Trailing whitespace (trailing-whitespace)
game_of_life.py:136:0: C0304: Final newline missing (missing-final-newline)
game_of_life.py:80:0: R1707: Disallow trailing comma tuple (trailing-comma-tuple)
game_of_life.py:

In [18]:
#! pip install autopep8
! autopep8 -i game_of_life.py

In [19]:
! pylint game_of_life.py

************* Module game_of_life
game_of_life.py:85:0: R1707: Disallow trailing comma tuple (trailing-comma-tuple)
game_of_life.py:13:0: R0402: Use 'from matplotlib import animation' instead (consider-using-from-import)
game_of_life.py:20:0: C0103: Function name "randomGrid" doesn't conform to snake_case naming style (invalid-name)
game_of_life.py:20:15: C0103: Argument name "N" doesn't conform to snake_case naming style (invalid-name)
game_of_life.py:25:0: C0103: Function name "addGlider" doesn't conform to snake_case naming style (invalid-name)
game_of_life.py:33:0: C0103: Function name "addGosperGliderGun" doesn't conform to snake_case naming style (invalid-name)
game_of_life.py:62:0: C0116: Missing function or method docstring (missing-function-docstring)
game_of_life.py:62:11: C0103: Argument name "frameNum" doesn't conform to snake_case naming style (invalid-name)
game_of_life.py:62:32: C0103: Argument name "N" doesn't conform to snake_case naming style (invalid-name)
game_of_li

In [15]:
# Dont need to run this again
# sphinx-quickstart
# sphinx-build -M html source build
# TO UPDATE THE HTML FILE
# make html